<a href="https://colab.research.google.com/github/asraj/END/blob/main/END_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Team: Stephen Raj A, Atul Gupta, Kranti & Rajeswar Reddy

3 LSTM layers are added using dynamically by fixing the first layer and then adding n-1 (2) more layers dynamically in a for loop

For reversing the training text, each batch of text sent for training is reversed using torch.flip() method


In [2]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:30, 930kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 56.2MB/s]


In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [223]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

Build the Model
The number of layers us build by fixing the first LSTM layer and then dynamically adding n-1 layers

In [258]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # First layer LSTM
        self.rnn1 = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=1, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.rnns = nn.ModuleList([nn.LSTM(hidden_dim, 
                           hidden_dim, 
                           num_layers=1, 
                           bidirectional=bidirectional, 
                           dropout=dropout) for i in range(n_layers-1)])
        
        
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        self.n_layers =n_layers
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #print(f'embedded size:{embedded.shape}')
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())

        #
        #print(f'Packed_embedded size:{packed_embedded.data.shape}')
        
        

        # First layer
        packed_output, (hidden, cell) = self.rnn1(packed_embedded)
        
        for i in range(self.n_layers-1):
          packed_output, (hidden, cell) = self.rnns[i](hidden)


        #print(f'packed_output3 size: {packed_output3.data.shape}, hidden size:{hidden.shape}, cell size:{cell.shape}')
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output3)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [259]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [260]:
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn1): LSTM(100, 256, dropout=0.2, bidirectional=True)
  (rnns): ModuleList(
    (0): LSTM(256, 256, dropout=0.2, bidirectional=True)
    (1): LSTM(256, 256, dropout=0.2, bidirectional=True)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [261]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,339,241 trainable parameters


In [262]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [263]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.5176,  1.6721,  0.1763,  ...,  1.0375,  0.3338, -0.7899],
        [-0.6318, -0.5060, -1.2920,  ...,  0.2655, -0.1766,  1.8327],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [264]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


Train the Model

In [265]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [266]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [267]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [274]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        #print(text, text.shape)

        #Reverse the text using torch.flip
        text = torch.flip(text, [0])

        #print(text, text.shape)
        
        #break
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [269]:
x = torch.arange(8).view(2, 4)
x

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])

In [270]:
torch.flip(x,[1])

tensor([[3, 2, 1, 0],
        [7, 6, 5, 4]])

In [253]:
torch.flip(x,[0])

tensor([[4, 5, 6, 7],
        [0, 1, 2, 3]])

In [271]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [272]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [275]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 15s
	Train Loss: 0.683 | Train Acc: 55.63%
	 Val. Loss: 0.628 |  Val. Acc: 67.94%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.441 | Train Acc: 80.10%
	 Val. Loss: 0.323 |  Val. Acc: 86.61%
Epoch: 03 | Epoch Time: 0m 15s
	Train Loss: 0.255 | Train Acc: 90.13%
	 Val. Loss: 0.307 |  Val. Acc: 87.69%
Epoch: 04 | Epoch Time: 0m 15s
	Train Loss: 0.182 | Train Acc: 93.41%
	 Val. Loss: 0.307 |  Val. Acc: 88.29%
Epoch: 05 | Epoch Time: 0m 15s
	Train Loss: 0.142 | Train Acc: 95.19%
	 Val. Loss: 0.365 |  Val. Acc: 88.17%


In [276]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.323 | Test Acc: 86.61%


In [277]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [278]:
predict_sentiment(model, "This film is terrible")

0.06398766487836838

In [279]:
predict_sentiment(model, "terrible is film This")

0.020517244935035706

In [280]:
predict_sentiment(model, "This film is great")

0.9674586057662964

In [281]:
predict_sentiment(model, "great is film This")

0.9859662652015686